# Day 19: Aplenty

[*Advent of Code 2023 day 19*](https://adventofcode.com/2023/day/19) and [*solution megathread*](https://redd.it/18ltr8m)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/19/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F19%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
part1_example_input = '''px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}'''

In [5]:
from typing import Tuple, Callable, NamedTuple
from operator import gt, lt


class RuleExpression(NamedTuple):
    field: str
    op: Callable
    threshold: int
    value_if_true: str

    @staticmethod
    def new_from_strings(field: str, op: str, threshold: str, value_if_true: str):
        match op:
            case '>':
                op_func = gt
            case '<':
                op_func = lt
            case _:
                raise ValueError(f"Expected less than (<) or greater than (>): {op=}")
        threshold_int = int(threshold)
        return RuleExpression(
            field=field,
            op=op_func,
            threshold=threshold_int,
            value_if_true=value_if_true)

class Rule(NamedTuple):
    name: str
    expressions: Tuple[RuleExpression, ...]
    default: str

class Part(NamedTuple):
    x: int
    m: int
    a: int
    s: int

In [8]:
from typing import Dict, List, Iterable
import re


def parse_input(input: str) -> Tuple[Dict[str, Rule], List[Part]]:
    def iter_ruleexpressions(expressions_str: str) -> Iterable[RuleExpression]:
        return map(
            lambda re_match: RuleExpression.new_from_strings(**re_match.groupdict()),
            re.finditer(
                r'(?P<field>[xmas])(?P<op>[><])(?P<threshold>\d+):(?P<value_if_true>[a-z]+|[RA]),',
                expressions_str
                )
            )
    def parse_rule(rule: str) -> Rule:
        name, expressions_str, default = re.findall(r'^([a-z]+){(.+?,)([a-z]+|[RA])}$', rule)[0]
        ruleexpressions = tuple(iter_ruleexpressions(expressions_str))
        return Rule(name=name, expressions=ruleexpressions, default=default)
    def parse_part(part: str) -> Part:
        # Dropping opening and closing brace, fields are in "xmas" order, so we
        # can split by comma and drop "[xmas]=" of each field
        return Part(*(int(p[2:]) for p in part[1:-1].split(',')))
    
    
    rules_str, parts_str = map(str.splitlines, input.split('\n\n'))
    rules = dict({rule.name: rule for rule in map(parse_rule, rules_str)})
    parts = list(map(parse_part, parts_str))
    return rules, parts

rules, parts = parse_input(part1_example_input)
print('\n'.join(map(str, rules.items())))
print('\n'.join(map(str, parts)))
# rules, parts = parse_input(downloaded['input'])

('px', Rule(name='px', expressions=(RuleExpression(field='a', op=<built-in function lt>, threshold=2006, value_if_true='qkq'), RuleExpression(field='m', op=<built-in function gt>, threshold=2090, value_if_true='A')), default='rfg'))
('pv', Rule(name='pv', expressions=(RuleExpression(field='a', op=<built-in function gt>, threshold=1716, value_if_true='R'),), default='A'))
('lnx', Rule(name='lnx', expressions=(RuleExpression(field='m', op=<built-in function gt>, threshold=1548, value_if_true='A'),), default='A'))
('rfg', Rule(name='rfg', expressions=(RuleExpression(field='s', op=<built-in function lt>, threshold=537, value_if_true='gd'), RuleExpression(field='x', op=<built-in function gt>, threshold=2440, value_if_true='R')), default='A'))
('qs', Rule(name='qs', expressions=(RuleExpression(field='s', op=<built-in function gt>, threshold=3448, value_if_true='A'),), default='lnx'))
('qkq', Rule(name='qkq', expressions=(RuleExpression(field='x', op=<built-in function lt>, threshold=1416, va

In [9]:
accepted = []
for part in parts:
    print(part)
    next_rule = 'in'
    while next_rule not in 'RA':
        rule = rules[next_rule]
        print(rule)
        for expression in rule.expressions:
            # First class functions for the win! op (and threshold) are parsed
            # from strings in the static method
            if expression.op(
                    part.__getattribute__(expression.field),
                    expression.threshold):
                next_rule = expression.value_if_true 
                break
        # Using for... else here feels like just snobbery, could as well have 
        # placed the default before the for loop
        else:
            next_rule = rule.default
    if next_rule == 'A':
        accepted.append(part)

# See how elegant it is to sum NamedTuple (but namedtuple.__getattribute__(attribute)
# is less elegant than dictionary[attribute])
sum(sum(part) for part in accepted)

Part(x=787, m=2655, a=1222, s=2876)
Rule(name='in', expressions=(RuleExpression(field='s', op=<built-in function lt>, threshold=1351, value_if_true='px'),), default='qqz')
Rule(name='qqz', expressions=(RuleExpression(field='s', op=<built-in function gt>, threshold=2770, value_if_true='qs'), RuleExpression(field='m', op=<built-in function lt>, threshold=1801, value_if_true='hdj')), default='R')
Rule(name='qs', expressions=(RuleExpression(field='s', op=<built-in function gt>, threshold=3448, value_if_true='A'),), default='lnx')
Rule(name='lnx', expressions=(RuleExpression(field='m', op=<built-in function gt>, threshold=1548, value_if_true='A'),), default='A')
Part(x=1679, m=44, a=2067, s=496)
Rule(name='in', expressions=(RuleExpression(field='s', op=<built-in function lt>, threshold=1351, value_if_true='px'),), default='qqz')
Rule(name='px', expressions=(RuleExpression(field='a', op=<built-in function lt>, threshold=2006, value_if_true='qkq'), RuleExpression(field='m', op=<built-in funct

19114

In [10]:
HTML(downloaded['part2'])